In [2]:
!pip install mediapipe vosk sounddevice scipy opencv-python numpy playsound
# Download a Vosk English model (40 MB) once:\

  Using cached mediapipe-0.10.21-cp311-cp311-macosx_11_0_universal2.whl.metadata (9.9 kB)
  Using cached vosk-0.3.44-py3-none-macosx_10_6_universal2.whl.metadata (1.8 kB)
  Using cached sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl.metadata (1.4 kB)
  Using cached jax-0.6.0-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.6.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (1.2 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
  Using cached sentencepiece-0.2.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached srt-3.5.3.tar.gz (28 kB)
  Preparing metadata (setup.py) ... done
  Using cached websockets-15.0.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (21 kB)
Using cached mediapipe-0.10.21-cp311-cp311-macosx_11_0_universal2.whl (49.2 MB)
Using cached vosk-0.3.44-py3-none-macosx_10_6_universal2.whl (4.7 MB

In [1]:
#!/usr/bin/env python3
"""
Driver Safety Suite (final)
--------------------------------
* Steering direction (green sticker)
* Drowsiness (eyes closed > 2 s)
* Drunk / unwell  – head bowed > 1.5 s **or** shake RMS > 60 px sustained ≥ 3 s
* Stress – shouting > 2 s **or** keywords "FUCK", "MOVE ASIDE" (offline ASR)

Controls
========
SPACE  – calibrate steering centre
S      - manually trigger stress (test)
B      - test beep sound
Q      – quit
"""

import cv2, mediapipe as mp, numpy as np, time, threading, queue, json, collections, subprocess, os, sys
import sounddevice as sd
from playsound import playsound
try:
    from vosk import Model, KaldiRecognizer
    VOSK_AVAILABLE = True
except ImportError:
    print("WARNING: Vosk speech recognition not available")
    VOSK_AVAILABLE = False

# ---------------------------- CONFIG ------------------------------------
LOWER = np.array([40,  80,  80])      # HSV range for neon‑green sticker
UPPER = np.array([85, 255, 255])
STEER_DEAD_PX    = 60                 # half‑width of steering dead‑zone (px)
STEER_FRAMES_REQ = 4                  # frames beyond dead‑zone before state flips

EAR_TH        = 0.18                  # eye‑aspect‑ratio threshold
DROWSY_SEC    = 2.0

BOW_SEC       = 1.5
SHAKE_RMS_PX  = 60
SHAKE_SEC     = 3.0

# Substantially lower thresholds for stress detection
SHOUT_GAIN    = 10.0                   # Volume threshold multiplier
SHOUT_SEC     = 1.0                   # Seconds required for shouting
STRESS_DISPLAY_SEC = 5.0              # How long to display the stress warning

KW_SET        = {"FUCK"}              # Individual keywords
KW_PHRASES    = ["MOVE ASIDE"]        # Phrases to detect

AUDIO_BLOCK   = 0.5                   # Half-second blocks for quicker response
BEEP_PATH     = "/Users/samridhgirdhar/Downloads/beep2.mp3"            # Default beep sound
MODEL_DIR     = "vosk_en"             # path to Vosk EN model directory
# ------------------------------------------------------------------------

# ---------------------------- HELPERS -----------------------------------
mp_face = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
LIDS_L  = [33,160,158,133,153,144]
LIDS_R  = [362,385,387,263,373,380]

def find_blob(mask):
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None
    c = max(cnts, key=cv2.contourArea)
    M = cv2.moments(c)
    if M["m00"] == 0:
        return None
    return int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])

def ear(pts, idx):
    """Compute eye‑aspect‑ratio for given 6‑point lid indices."""
    p2p6 = np.linalg.norm(pts[idx[1]] - pts[idx[5]])
    p3p5 = np.linalg.norm(pts[idx[2]] - pts[idx[4]])
    p1p4 = np.linalg.norm(pts[idx[0]] - pts[idx[3]])
    return (p2p6 + p3p5) / (2.0 * p1p4)

def play_beep():
    """Simple beep function that works across platforms."""
    print("\a", end="", flush=True)  # System bell as default
    
    try:
        # Use platform-specific commands for more reliable sound
        if sys.platform == 'win32':
            import winsound
            winsound.Beep(1000, 500)
        elif sys.platform == 'darwin':  # macOS
            subprocess.run(['afplay', '/System/Library/Sounds/Tink.aiff'], 
                          check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        elif sys.platform == 'linux':
            subprocess.run(['paplay', '/usr/share/sounds/freedesktop/stereo/complete.oga'],
                          check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception:
        pass  # Fallback to system bell already done

# ----------------------- AUDIO / STRESS DETECTION --------------------------

def setup_vosk(sample_rate=16000):
    """Set up Vosk speech recognition if available."""
    if not VOSK_AVAILABLE:
        return None
        
    try:
        model = Model(MODEL_DIR) 
        return KaldiRecognizer(model, sample_rate)
    except Exception as e:
        print(f"ERROR setting up Vosk: {e}")
        return None

def process_audio(stress_callback):
    """Process audio in a separate thread and detect stress via shouting or keywords."""
    fs = 16000
    
    # Try to set up speech recognition
    recognizer = setup_vosk(fs)
    speech_recognition = recognizer is not None
    
    print(f"Audio processing thread started. Speech recognition: {speech_recognition}")
    
    # Energy tracking variables
    audio_energies = collections.deque(maxlen=20)  # ~10 seconds history
    high_energy_start = None
    baseline_multiplier = 1.0  # Start with normal sensitivity
    
    # Initial baseline calculation period
    print("Calculating audio baseline... please stay quiet")
    for _ in range(5):  # ~2.5 seconds to establish baseline
        audio = sd.rec(int(AUDIO_BLOCK * fs), samplerate=fs, channels=1, dtype='float32')
        sd.wait()
        block = audio.flatten()
        energy = np.sqrt(np.mean(block ** 2))
        audio_energies.append(energy)
        time.sleep(0.1)
    
    baseline_energy = np.mean(audio_energies) * 1.2  # Add 20% margin
    print(f"Audio baseline established: {baseline_energy:.6f}")
    
    # Main loop
    while True:
        try:
            # Record audio
            audio = sd.rec(int(AUDIO_BLOCK * fs), samplerate=fs, channels=1, dtype='float32')
            sd.wait()
            block = audio.flatten()
            
            # Calculate energy and update rolling history
            energy = np.sqrt(np.mean(block ** 2))
            audio_energies.append(energy)
            
            # Gradually update baseline to adapt to environment
            if len(audio_energies) >= 10:  # Wait for enough history
                # Use the 20th percentile as the baseline for better stability
                sorted_energies = sorted(audio_energies)
                baseline_energy = sorted_energies[int(len(sorted_energies) * 0.2)]
            
            # Print energy levels occasionally for debugging
            if np.random.random() < 0.05:  # ~5% of blocks
                energy_ratio = energy / baseline_energy
                print(f"Audio: energy={energy:.6f}, baseline={baseline_energy:.6f}, ratio={energy_ratio:.2f}")
            
            # Shouting detection
            if energy > baseline_energy * SHOUT_GAIN * baseline_multiplier:
                if high_energy_start is None:
                    high_energy_start = time.time()
                    print(f"High volume detected! Ratio: {energy/baseline_energy:.2f}")
            else:
                high_energy_start = None
            
            # If high energy sustains long enough, trigger stress
            if high_energy_start and (time.time() - high_energy_start) >= SHOUT_SEC:
                print("STRESS DETECTED: SHOUTING")
                stress_callback()
                high_energy_start = None
                # Temporarily increase sensitivity for next few seconds
                baseline_multiplier = 0.8
                time.sleep(1.0)  # Pause briefly to avoid rapid retriggers
                continue
            
            # Restore normal sensitivity over time
            baseline_multiplier = min(1.0, baseline_multiplier + 0.01)
            
            # Speech recognition for keywords
            if speech_recognition:
                # Convert float32 → int16 PCM for Vosk
                block_int16 = (block * 32767).astype(np.int16).tobytes()
                
                if recognizer.AcceptWaveform(block_int16):
                    result = json.loads(recognizer.Result())
                    transcript = result.get("text", "").upper()
                    
                    if transcript:
                        print(f"Speech: {transcript}")
                        
                        # Check for keywords
                        found_keywords = [w for w in transcript.split() if w in KW_SET]
                        
                        # Check for phrases
                        found_phrases = [p for p in KW_PHRASES if p in transcript]
                        
                        if found_keywords:
                            print(f"STRESS DETECTED: Keywords {found_keywords}")
                            stress_callback()
                        
                        if found_phrases:
                            print(f"STRESS DETECTED: Phrase {found_phrases}")
                            stress_callback()
        
        except KeyboardInterrupt:
            break
        except Exception as e:
            print(f"Audio error: {e}")
            time.sleep(1)  # Avoid error spam

# ----------------------------- MAIN -------------------------------------

def main(cam_index: int = 0):
    try:
        # Initialize camera
        cap = cv2.VideoCapture(cam_index)
        if not cap.isOpened():
            print("Camera not found - trying default camera")
            cap = cv2.VideoCapture(0)
            if not cap.isOpened():
                raise RuntimeError("No cameras found - please connect a webcam or enable camera access")

        # ------- steering state -------
        cx0 = None
        steer_state = "STRAIGHT"
        consecutive_lr = 0

        # ------- timers & buffers -------
        eye_closed_start = None
        head_bow_start   = None
        shake_start      = None
        shake_buf        = collections.deque(maxlen=30)  # ≈1 s at 30 fps
        last_beep = 0
        
        # ------- stress state -------
        stressed = False
        stress_end_time = 0
        
        def trigger_stress():
            nonlocal stressed, stress_end_time
            stressed = True
            stress_end_time = time.time() + STRESS_DISPLAY_SEC
            play_beep()
        
        # Start audio processing in background thread
        audio_thread = threading.Thread(
            target=process_audio,
            args=(trigger_stress,),
            daemon=True
        )
        audio_thread.start()

        # Wait a moment for audio thread to initialize
        time.sleep(2)
        print("Driver Safety Suite started. Press SPACE to calibrate steering, Q to quit.")

        with mp_face.FaceMesh(max_num_faces=1, refine_landmarks=True,
                            min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh, \
            mp_pose.Pose(static_image_mode=False) as pose:
            
            while True:
                # Get frame from camera
                ok, frame = cap.read()
                if not ok:
                    print("Camera read failed")
                    break
                
                h, w = frame.shape[:2]
                
                # Reset stress flag if timeout elapsed
                if stressed and time.time() > stress_end_time:
                    stressed = False

                # ================== Steering detection ==================
                hsv  = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv, LOWER, UPPER)
                mask = cv2.erode(mask, None, 2)
                mask = cv2.dilate(mask, None, 2)
                blob = find_blob(mask)

                if blob and cx0 is not None:
                    dx = blob[0] - cx0
                    if abs(dx) > STEER_DEAD_PX:
                        consecutive_lr += 1
                        if consecutive_lr >= STEER_FRAMES_REQ:
                            steer_state = "LEFT" if dx < 0 else "RIGHT"
                            consecutive_lr = STEER_FRAMES_REQ
                    else:
                        consecutive_lr = 0
                        steer_state = "STRAIGHT"
                elif not blob:
                    steer_state = "STKR MISS"
                else:
                    steer_state = "UN CAL"

                cv2.putText(frame, steer_state, (20, 60), cv2.FONT_HERSHEY_DUPLEX, 1.2, (255, 255, 255), 3)

                # ================== Face & pose landmarks ==================
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                face_res = face_mesh.process(rgb)
                pose_res = pose.process(rgb)

                drowsy = False
                drunk  = False

                # ---------- drowsiness via eyes ----------
                if face_res.multi_face_landmarks:
                    pts = np.array([(p.x * w, p.y * h) for p in face_res.multi_face_landmarks[0].landmark])
                    ear_now = (ear(pts, LIDS_L) + ear(pts, LIDS_R)) / 2

                    if ear_now < EAR_TH:
                        eye_closed_start = eye_closed_start or time.time()
                    else:
                        eye_closed_start = None

                    if eye_closed_start and time.time() - eye_closed_start >= DROWSY_SEC:
                        drowsy = True

                    # ---------- head bow ----------
                    eye_y  = pts[1][1]
                    chin_y = pts[152][1]
                    bowed = (eye_y - chin_y) > 40
                    if bowed:
                        head_bow_start = head_bow_start or time.time()
                    else:
                        head_bow_start = None
                    if head_bow_start and time.time() - head_bow_start >= BOW_SEC:
                        drunk = True

                # ---------- shake detection ----------
                if pose_res.pose_landmarks:
                    nose_x = pose_res.pose_landmarks.landmark[0].x * w
                    shake_buf.append(nose_x)
                    if len(shake_buf) == shake_buf.maxlen:
                        rms = np.std(shake_buf)
                        if rms > SHAKE_RMS_PX:
                            shake_start = shake_start or time.time()
                        else:
                            shake_start = None

                        if shake_start and time.time() - shake_start >= SHAKE_SEC:
                            drunk = True
                else:
                    shake_buf.clear()
                    shake_start = None

                # ================== Alerts & UI ==================
                alerts = []
                if drowsy:
                    alerts.append("DROWSY")
                if drunk:
                    alerts.append("DRUNK/UNWELL")
                if stressed:
                    alerts.append("STRESS")

                for i, txt in enumerate(alerts):
                    cv2.putText(frame, txt, (20, 120 + i * 50), 
                                cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 0, 255), 3)

                if alerts and time.time() - last_beep > 0.8:
                    play_beep()
                    last_beep = time.time()
                    print(*alerts, sep=" | ")

                # Add mic status visual indicator
                mic_status = "MIC ON" if audio_thread.is_alive() else "MIC ERROR"
                cv2.putText(frame, mic_status, (w-150, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                            (0, 255, 0) if audio_thread.is_alive() else (0, 0, 255), 2)

                # ================== Key handling & display ==================
                cv2.imshow("Driver Safety Suite", frame)
                key = cv2.waitKey(1) & 0xFF
                if key == ord(' '):
                    if blob:
                        cx0 = blob[0]
                        consecutive_lr = 0
                        print("Steering centre calibrated at", cx0)
                    else:
                        print("No sticker detected for calibration")
                elif key == ord('s'):  # Force stress test
                    trigger_stress()
                    print("Stress manually triggered")
                elif key == ord('b'):  # Test beep
                    play_beep()
                    print("Beep test")
                elif key == ord('q'):
                    break

        cap.release()
        cv2.destroyAllWindows()
        
    except Exception as e:
        print(f"ERROR: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()    


KeyboardInterrupt: 

In [1]:
#!/usr/bin/env python3
"""
Driver Safety Suite (final) with Arduino Communication
--------------------------------
* Steering direction (green sticker)
* Drowsiness (eyes closed > 2 s)
* Drunk / unwell  – head bowed > 1.5 s **or** shake RMS > 60 px sustained ≥ 3 s
* Stress – shouting > 2 s **or** keywords "FUCK", "MOVE ASIDE" (offline ASR)

Controls
========
SPACE  – calibrate steering centre
S      - manually trigger stress (test)
B      - test beep sound
Q      – quit
"""

import cv2, mediapipe as mp, numpy as np, time, threading, queue, json, collections, subprocess, os, sys
import sounddevice as sd
from playsound import playsound
import serial  # Added for Arduino communication

try:
    from vosk import Model, KaldiRecognizer
    VOSK_AVAILABLE = True
except ImportError:
    print("WARNING: Vosk speech recognition not available")
    VOSK_AVAILABLE = False

# ---------------------------- CONFIG ------------------------------------
LOWER = np.array([40,  80,  80])      # HSV range for neon‑green sticker
UPPER = np.array([85, 255, 255])
STEER_DEAD_PX    = 60                 # half‑width of steering dead‑zone (px)
STEER_FRAMES_REQ = 4                  # frames beyond dead‑zone before state flips

EAR_TH        = 0.18                  # eye‑aspect‑ratio threshold
DROWSY_SEC    = 2.0

BOW_SEC       = 1.5
SHAKE_RMS_PX  = 60
SHAKE_SEC     = 3.0

# Substantially lower thresholds for stress detection
SHOUT_GAIN    = 10.0                  # Volume threshold multiplier
SHOUT_SEC     = 1.0                   # Seconds required for shouting
STRESS_DISPLAY_SEC = 5.0              # How long to display the stress warning

KW_SET        = {"FUCK"}              # Individual keywords
KW_PHRASES    = ["MOVE ASIDE"]        # Phrases to detect

AUDIO_BLOCK   = 0.5                   # Half-second blocks for quicker response
BEEP_PATH     = "/Users/samridhgirdhar/Downloads/beep2.mp3"  # Default beep sound
MODEL_DIR     = "vosk_en"             # path to Vosk EN model directory

# Arduino Serial Communication Configuration
SERIAL_PORT = "/dev/cu.usbmodem2101"   # Arduino USB port - CHANGE THIS TO YOUR PORT
BAUD_RATE = 9600
SERIAL_TIMEOUT = 0.1                  # Short timeout for non-blocking
# ------------------------------------------------------------------------

# ---------------------------- HELPERS -----------------------------------
mp_face = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
LIDS_L  = [33,160,158,133,153,144]
LIDS_R  = [362,385,387,263,373,380]

def find_blob(mask):
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None
    c = max(cnts, key=cv2.contourArea)
    M = cv2.moments(c)
    if M["m00"] == 0:
        return None
    return int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])

def ear(pts, idx):
    """Compute eye‑aspect‑ratio for given 6‑point lid indices."""
    p2p6 = np.linalg.norm(pts[idx[1]] - pts[idx[5]])
    p3p5 = np.linalg.norm(pts[idx[2]] - pts[idx[4]])
    p1p4 = np.linalg.norm(pts[idx[0]] - pts[idx[3]])
    return (p2p6 + p3p5) / (2.0 * p1p4)

def play_beep():
    """Simple beep function that works across platforms."""
    print("\a", end="", flush=True)  # System bell as default
    
    try:
        # Use platform-specific commands for more reliable sound
        if sys.platform == 'win32':
            import winsound
            winsound.Beep(1000, 500)
        elif sys.platform == 'darwin':  # macOS
            subprocess.run(['afplay', '/System/Library/Sounds/Tink.aiff'], 
                          check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        elif sys.platform == 'linux':
            subprocess.run(['paplay', '/usr/share/sounds/freedesktop/stereo/complete.oga'],
                          check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception:
        pass  # Fallback to system bell already done

# Function to send data to Arduino
def send_to_arduino(ser, message):
    """Send a message to Arduino with newline termination."""
    if ser and ser.is_open:
        try:
            ser.write((message + "\n").encode())
            print(f"[ARDUINO] Sent: {message}")
        except Exception as e:
            print(f"[ARDUINO] Error sending data: {e}")

# ----------------------- AUDIO / STRESS DETECTION --------------------------

def setup_vosk(sample_rate=16000):
    """Set up Vosk speech recognition if available."""
    if not VOSK_AVAILABLE:
        return None
        
    try:
        model = Model(MODEL_DIR) 
        return KaldiRecognizer(model, sample_rate)
    except Exception as e:
        print(f"ERROR setting up Vosk: {e}")
        return None

def process_audio(stress_callback):
    """Process audio in a separate thread and detect stress via shouting or keywords."""
    fs = 16000
    
    # Try to set up speech recognition
    recognizer = setup_vosk(fs)
    speech_recognition = recognizer is not None
    
    print(f"Audio processing thread started. Speech recognition: {speech_recognition}")
    
    # Energy tracking variables
    audio_energies = collections.deque(maxlen=20)  # ~10 seconds history
    high_energy_start = None
    baseline_multiplier = 1.0  # Start with normal sensitivity
    
    # Initial baseline calculation period
    print("Calculating audio baseline... please stay quiet")
    for _ in range(5):  # ~2.5 seconds to establish baseline
        audio = sd.rec(int(AUDIO_BLOCK * fs), samplerate=fs, channels=1, dtype='float32')
        sd.wait()
        block = audio.flatten()
        energy = np.sqrt(np.mean(block ** 2))
        audio_energies.append(energy)
        time.sleep(0.1)
    
    baseline_energy = np.mean(audio_energies) * 1.2  # Add 20% margin
    print(f"Audio baseline established: {baseline_energy:.6f}")
    
    # Main loop
    while True:
        try:
            # Record audio
            audio = sd.rec(int(AUDIO_BLOCK * fs), samplerate=fs, channels=1, dtype='float32')
            sd.wait()
            block = audio.flatten()
            
            # Calculate energy and update rolling history
            energy = np.sqrt(np.mean(block ** 2))
            audio_energies.append(energy)
            
            # Gradually update baseline to adapt to environment
            if len(audio_energies) >= 10:  # Wait for enough history
                # Use the 20th percentile as the baseline for better stability
                sorted_energies = sorted(audio_energies)
                baseline_energy = sorted_energies[int(len(sorted_energies) * 0.2)]
            
            # Print energy levels occasionally for debugging
            if np.random.random() < 0.05:  # ~5% of blocks
                energy_ratio = energy / baseline_energy
                print(f"Audio: energy={energy:.6f}, baseline={baseline_energy:.6f}, ratio={energy_ratio:.2f}")
            
            # Shouting detection
            if energy > baseline_energy * SHOUT_GAIN * baseline_multiplier:
                if high_energy_start is None:
                    high_energy_start = time.time()
                    print(f"High volume detected! Ratio: {energy/baseline_energy:.2f}")
            else:
                high_energy_start = None
            
            # If high energy sustains long enough, trigger stress
            if high_energy_start and (time.time() - high_energy_start) >= SHOUT_SEC:
                print("STRESS DETECTED: SHOUTING")
                stress_callback()
                high_energy_start = None
                # Temporarily increase sensitivity for next few seconds
                baseline_multiplier = 0.8
                time.sleep(1.0)  # Pause briefly to avoid rapid retriggers
                continue
            
            # Restore normal sensitivity over time
            baseline_multiplier = min(1.0, baseline_multiplier + 0.01)
            
            # Speech recognition for keywords
            if speech_recognition:
                # Convert float32 → int16 PCM for Vosk
                block_int16 = (block * 32767).astype(np.int16).tobytes()
                
                if recognizer.AcceptWaveform(block_int16):
                    result = json.loads(recognizer.Result())
                    transcript = result.get("text", "").upper()
                    
                    if transcript:
                        print(f"Speech: {transcript}")
                        
                        # Check for keywords
                        found_keywords = [w for w in transcript.split() if w in KW_SET]
                        
                        # Check for phrases
                        found_phrases = [p for p in KW_PHRASES if p in transcript]
                        
                        if found_keywords:
                            print(f"STRESS DETECTED: Keywords {found_keywords}")
                            stress_callback()
                        
                        if found_phrases:
                            print(f"STRESS DETECTED: Phrase {found_phrases}")
                            stress_callback()
        
        except KeyboardInterrupt:
            break
        except Exception as e:
            print(f"Audio error: {e}")
            time.sleep(1)  # Avoid error spam

# ----------------------------- MAIN -------------------------------------

def main(cam_index: int = 1):
    try:
        # Initialize camera
        cap = cv2.VideoCapture(cam_index)
        if not cap.isOpened():
            print("Camera not found - trying default camera")
            cap = cv2.VideoCapture(0)
            if not cap.isOpened():
                raise RuntimeError("No cameras found - please connect a webcam or enable camera access")

        # Initialize serial connection to Arduino
        ser = None
        try:
            ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=SERIAL_TIMEOUT)
            print(f"[ARDUINO] Connected to {SERIAL_PORT} at {BAUD_RATE} baud")
            # Give Arduino time to reset after connection
            time.sleep(2)
            # Send initial message to test connection
            send_to_arduino(ser, "SYSTEM:READY")
        except Exception as e:
            print(f"[ARDUINO] Connection failed: {e}")
            print("[ARDUINO] Continuing without Arduino communication")

        # ------- steering state -------
        cx0 = None
        steer_state = "STRAIGHT"
        last_steer_state = None
        consecutive_lr = 0

        # ------- timers & buffers -------
        eye_closed_start = None
        head_bow_start   = None
        shake_start      = None
        shake_buf        = collections.deque(maxlen=30)  # ≈1 s at 30 fps
        last_beep = 0
        
        # ------- state tracking -------
        last_drowsy = False
        last_drunk = False
        
        # ------- stress state -------
        stressed = False
        last_stressed = False
        stress_end_time = 0
        
        def trigger_stress():
            nonlocal stressed, stress_end_time
            stressed = True
            stress_end_time = time.time() + STRESS_DISPLAY_SEC
            play_beep()
        
        # Start audio processing in background thread
        audio_thread = threading.Thread(
            target=process_audio,
            args=(trigger_stress,),
            daemon=True
        )
        audio_thread.start()

        # Wait a moment for audio thread to initialize
        time.sleep(2)
        print("Driver Safety Suite started. Press SPACE to calibrate steering, Q to quit.")

        with mp_face.FaceMesh(max_num_faces=1, refine_landmarks=True,
                            min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh, \
            mp_pose.Pose(static_image_mode=False) as pose:
            
            while True:
                # Get frame from camera
                ok, frame = cap.read()
                if not ok:
                    print("Camera read failed")
                    break
                
                h, w = frame.shape[:2]
                
                # Reset stress flag if timeout elapsed
                if stressed and time.time() > stress_end_time:
                    stressed = False

                # ================== Steering detection ==================
                hsv  = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv, LOWER, UPPER)
                mask = cv2.erode(mask, None, 2)
                mask = cv2.dilate(mask, None, 2)
                blob = find_blob(mask)

                if blob and cx0 is not None:
                    dx = blob[0] - cx0
                    if abs(dx) > STEER_DEAD_PX:
                        consecutive_lr += 1
                        if consecutive_lr >= STEER_FRAMES_REQ:
                            steer_state = "LEFT" if dx < 0 else "RIGHT"
                            consecutive_lr = STEER_FRAMES_REQ
                    else:
                        consecutive_lr = 0
                        steer_state = "STRAIGHT"
                elif not blob:
                    steer_state = "STKR MISS"
                else:
                    steer_state = "UN CAL"

                cv2.putText(frame, steer_state, (20, 60), cv2.FONT_HERSHEY_DUPLEX, 1.2, (255, 255, 255), 3)

                # Send steering state to Arduino if changed
                if steer_state != last_steer_state and steer_state in ("LEFT", "RIGHT", "STRAIGHT"):
                    send_to_arduino(ser, f"STEER:{steer_state}")
                last_steer_state = steer_state

                # ================== Face & pose landmarks ==================
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                face_res = face_mesh.process(rgb)
                pose_res = pose.process(rgb)

                drowsy = False
                drunk  = False

                # ---------- drowsiness via eyes ----------
                if face_res.multi_face_landmarks:
                    pts = np.array([(p.x * w, p.y * h) for p in face_res.multi_face_landmarks[0].landmark])
                    ear_now = (ear(pts, LIDS_L) + ear(pts, LIDS_R)) / 2

                    if ear_now < EAR_TH:
                        eye_closed_start = eye_closed_start or time.time()
                    else:
                        eye_closed_start = None

                    if eye_closed_start and time.time() - eye_closed_start >= DROWSY_SEC:
                        drowsy = True

                    # ---------- head bow ----------
                    eye_y  = pts[1][1]
                    chin_y = pts[152][1]
                    bowed = (eye_y - chin_y) > 40
                    if bowed:
                        head_bow_start = head_bow_start or time.time()
                    else:
                        head_bow_start = None
                    if head_bow_start and time.time() - head_bow_start >= BOW_SEC:
                        drunk = True

                # ---------- shake detection ----------
                if pose_res.pose_landmarks:
                    nose_x = pose_res.pose_landmarks.landmark[0].x * w
                    shake_buf.append(nose_x)
                    if len(shake_buf) == shake_buf.maxlen:
                        rms = np.std(shake_buf)
                        if rms > SHAKE_RMS_PX:
                            shake_start = shake_start or time.time()
                        else:
                            shake_start = None

                        if shake_start and time.time() - shake_start >= SHAKE_SEC:
                            drunk = True
                else:
                    shake_buf.clear()
                    shake_start = None

                # ================== Send state changes to Arduino ==================
                # Send drowsy state change
                if drowsy != last_drowsy:
                    if drowsy:
                        send_to_arduino(ser, "ALERT:DROWSY")
                    else:
                        send_to_arduino(ser, "CLEAR:DROWSY")
                last_drowsy = drowsy
                
                # Send drunk/unwell state change
                if drunk != last_drunk:
                    if drunk:
                        send_to_arduino(ser, "ALERT:DRUNK")
                    else:
                        send_to_arduino(ser, "CLEAR:DRUNK")
                last_drunk = drunk
                
                # Send stress state change
                if stressed != last_stressed:
                    if stressed:
                        send_to_arduino(ser, "ALERT:STRESS")
                    else:
                        send_to_arduino(ser, "CLEAR:STRESS")
                last_stressed = stressed

                # ================== Alerts & UI ==================
                alerts = []
                if drowsy:
                    alerts.append("DROWSY")
                if drunk:
                    alerts.append("DRUNK/UNWELL")
                if stressed:
                    alerts.append("STRESS")

                for i, txt in enumerate(alerts):
                    cv2.putText(frame, txt, (20, 120 + i * 50), 
                                cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 0, 255), 3)

                if alerts and time.time() - last_beep > 0.8:
                    play_beep()
                    last_beep = time.time()
                    print(*alerts, sep=" | ")

                # Add mic status visual indicator
                mic_status = "MIC ON" if audio_thread.is_alive() else "MIC ERROR"
                cv2.putText(frame, mic_status, (w-150, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                            (0, 255, 0) if audio_thread.is_alive() else (0, 0, 255), 2)
                
                # Add Arduino status indicator
                arduino_status = "ARDUINO ON" if (ser and ser.is_open) else "ARDUINO OFF"
                cv2.putText(frame, arduino_status, (w-150, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                            (0, 255, 0) if (ser and ser.is_open) else (0, 0, 255), 2)

                # ================== Key handling & display ==================
                cv2.imshow("Driver Safety Suite", frame)
                key = cv2.waitKey(1) & 0xFF
                if key == ord(' '):
                    if blob:
                        cx0 = blob[0]
                        consecutive_lr = 0
                        print("Steering centre calibrated at", cx0)
                        send_to_arduino(ser, "SYSTEM:CALIBRATED")
                    else:
                        print("No sticker detected for calibration")
                elif key == ord('s'):  # Force stress test
                    trigger_stress()
                    print("Stress manually triggered")
                elif key == ord('b'):  # Test beep
                    play_beep()
                    print("Beep test")
                elif key == ord('q'):
                    # Send shutdown message to Arduino
                    send_to_arduino(ser, "SYSTEM:SHUTDOWN")
                    break

        # Clean up
        if ser and ser.is_open:
            ser.close()
            print("[ARDUINO] Connection closed")
        
        cap.release()
        cv2.destroyAllWindows()
        
    except Exception as e:
        print(f"ERROR: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()

KeyboardInterrupt: 

In [ ]:
#!/usr/bin/env python3
"""
Driver Safety Suite (complete) with Arduino Communication and AI Assistant Integration
------------------------------------------------------------------------------------
* Steering direction (green sticker)
* Drowsiness (eyes closed > 2 s)
* Drunk / unwell  – head bowed > 1.5 s **or** shake RMS > 60 px sustained ≥ 3 s
* Stress – shouting > 2 s **or** keywords "FUCK", "MOVE ASIDE" (offline ASR)

Controls
========
SPACE  – calibrate steering centre
S      - manually trigger stress (test)
B      - test beep sound
Q      – quit
"""

import cv2, mediapipe as mp, numpy as np, time, threading, queue, json, collections, subprocess, os, sys
import sounddevice as sd
from playsound import playsound
import serial  # For Arduino communication
import requests  # For AI assistant communication

try:
    from vosk import Model, KaldiRecognizer
    VOSK_AVAILABLE = True
except ImportError:
    print("WARNING: Vosk speech recognition not available")
    VOSK_AVAILABLE = False

# ---------------------------- CONFIG ------------------------------------
LOWER = np.array([40,  80,  80])      # HSV range for neon‑green sticker
UPPER = np.array([85, 255, 255])
STEER_DEAD_PX    = 60                 # half‑width of steering dead‑zone (px)
STEER_FRAMES_REQ = 4                  # frames beyond dead‑zone before state flips

EAR_TH        = 0.18                  # eye‑aspect‑ratio threshold
DROWSY_SEC    = 2.0

BOW_SEC       = 1.5
SHAKE_RMS_PX  = 60
SHAKE_SEC     = 3.0

# Substantially lower thresholds for stress detection
SHOUT_GAIN    = 15.0                  # Volume threshold multiplier
SHOUT_SEC     = 1.0                   # Seconds required for shouting
STRESS_DISPLAY_SEC = 5.0              # How long to display the stress warning

KW_SET        = {"FUCK"}              # Individual keywords
KW_PHRASES    = ["MOVE ASIDE"]        # Phrases to detect

AUDIO_BLOCK   = 0.5                   # Half-second blocks for quicker response
BEEP_PATH     = "/Users/samridhgirdhar/Downloads/beep2.mp3"  # Default beep sound
MODEL_DIR     = "vosk_en"             # path to Vosk EN model directory

# Arduino Serial Communication Configuration
SERIAL_PORT = "/dev/cu.usbmodem2101"   # Arduino USB port - CHANGE THIS TO YOUR PORT
BAUD_RATE = 9600
SERIAL_TIMEOUT = 0.1                  # Short timeout for non-blocking

# AI Assistant Communication Configuration
ASSISTANT_URL = "http://localhost:8080/alert"  # AI Assistant API endpoint
# ------------------------------------------------------------------------

# ---------------------------- HELPERS -----------------------------------
mp_face = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
LIDS_L  = [33,160,158,133,153,144]
LIDS_R  = [362,385,387,263,373,380]

def find_blob(mask):
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None
    c = max(cnts, key=cv2.contourArea)
    M = cv2.moments(c)
    if M["m00"] == 0:
        return None
    return int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])

def ear(pts, idx):
    """Compute eye‑aspect‑ratio for given 6‑point lid indices."""
    p2p6 = np.linalg.norm(pts[idx[1]] - pts[idx[5]])
    p3p5 = np.linalg.norm(pts[idx[2]] - pts[idx[4]])
    p1p4 = np.linalg.norm(pts[idx[0]] - pts[idx[3]])
    return (p2p6 + p3p5) / (2.0 * p1p4)

def play_beep():
    """Simple beep function that works across platforms."""
    print("\a", end="", flush=True)  # System bell as default
    
    try:
        # Use platform-specific commands for more reliable sound
        if sys.platform == 'win32':  # Windows
            import winsound
            winsound.Beep(1000, 500)
        elif sys.platform == 'darwin':  # macOS
            subprocess.run(['afplay', '/System/Library/Sounds/Tink.aiff'], 
                          check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        elif sys.platform == 'linux':
            subprocess.run(['paplay', '/usr/share/sounds/freedesktop/stereo/complete.oga'],
                          check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception:
        pass  # Fallback to system bell already done

# Function to send data to Arduino
def send_to_arduino(ser, message):
    """Send a message to Arduino with newline termination."""
    if ser and ser.is_open:
        try:
            ser.write((message + "\n").encode())
            print(f"[ARDUINO] Sent: {message}")
        except Exception as e:
            print(f"[ARDUINO] Error sending data: {e}")

# Function to send data to AI Assistant
def send_to_assistant(alert_type, state=True, additional_data=None):
    """Send an alert to the AI assistant."""
    try:
        data = {
            "type": alert_type,
            "state": state
        }
        
        if additional_data:
            data.update(additional_data)
            
        response = requests.post(ASSISTANT_URL, json=data, timeout=0.5)  # Short timeout to avoid blocking
        if response.status_code == 200:
            print(f"[ASSISTANT] Sent {alert_type} alert")
        else:
            print(f"[ASSISTANT] Failed to send alert: {response.status_code}")
    except Exception as e:
        print(f"[ASSISTANT] Error sending to assistant: {e}")

# ----------------------- AUDIO / STRESS DETECTION --------------------------

def setup_vosk(sample_rate=16000):
    """Set up Vosk speech recognition if available."""
    if not VOSK_AVAILABLE:
        return None
        
    try:
        model = Model(MODEL_DIR) 
        return KaldiRecognizer(model, sample_rate)
    except Exception as e:
        print(f"ERROR setting up Vosk: {e}")
        return None

def process_audio(stress_callback):
    """Process audio in a separate thread and detect stress via shouting or keywords."""
    fs = 16000
    
    # Try to set up speech recognition
    recognizer = setup_vosk(fs)
    speech_recognition = recognizer is not None
    
    print(f"Audio processing thread started. Speech recognition: {speech_recognition}")
    
    # Energy tracking variables
    audio_energies = collections.deque(maxlen=20)  # ~10 seconds history
    high_energy_start = None
    baseline_multiplier = 1.0  # Start with normal sensitivity
    
    # Initial baseline calculation period
    print("Calculating audio baseline... please stay quiet")
    for _ in range(5):  # ~2.5 seconds to establish baseline
        audio = sd.rec(int(AUDIO_BLOCK * fs), samplerate=fs, channels=1, dtype='float32')
        sd.wait()
        block = audio.flatten()
        energy = np.sqrt(np.mean(block ** 2))
        audio_energies.append(energy)
        time.sleep(0.1)
    
    baseline_energy = np.mean(audio_energies) * 1.2  # Add 20% margin
    print(f"Audio baseline established: {baseline_energy:.6f}")
    
    # Main loop
    while True:
        try:
            # Record audio
            audio = sd.rec(int(AUDIO_BLOCK * fs), samplerate=fs, channels=1, dtype='float32')
            sd.wait()
            block = audio.flatten()
            
            # Calculate energy and update rolling history
            energy = np.sqrt(np.mean(block ** 2))
            audio_energies.append(energy)
            
            # Gradually update baseline to adapt to environment
            if len(audio_energies) >= 10:  # Wait for enough history
                # Use the 20th percentile as the baseline for better stability
                sorted_energies = sorted(audio_energies)
                baseline_energy = sorted_energies[int(len(sorted_energies) * 0.2)]
            
            # Print energy levels occasionally for debugging
            if np.random.random() < 0.05:  # ~5% of blocks
                energy_ratio = energy / baseline_energy
                print(f"Audio: energy={energy:.6f}, baseline={baseline_energy:.6f}, ratio={energy_ratio:.2f}")
            
            # Shouting detection
            if energy > baseline_energy * SHOUT_GAIN * baseline_multiplier:
                if high_energy_start is None:
                    high_energy_start = time.time()
                    print(f"High volume detected! Ratio: {energy/baseline_energy:.2f}")
            else:
                high_energy_start = None
            
            # If high energy sustains long enough, trigger stress
            if high_energy_start and (time.time() - high_energy_start) >= SHOUT_SEC:
                print("STRESS DETECTED: SHOUTING")
                stress_callback()
                high_energy_start = None
                # Temporarily increase sensitivity for next few seconds
                baseline_multiplier = 0.8
                time.sleep(1.0)  # Pause briefly to avoid rapid retriggers
                continue
            
            # Restore normal sensitivity over time
            baseline_multiplier = min(1.0, baseline_multiplier + 0.01)
            
            # Speech recognition for keywords
            if speech_recognition:
                # Convert float32 → int16 PCM for Vosk
                block_int16 = (block * 32767).astype(np.int16).tobytes()
                
                if recognizer.AcceptWaveform(block_int16):
                    result = json.loads(recognizer.Result())
                    transcript = result.get("text", "").upper()
                    
                    if transcript:
                        print(f"Speech: {transcript}")
                        
                        # Check for keywords
                        found_keywords = [w for w in transcript.split() if w in KW_SET]
                        
                        # Check for phrases
                        found_phrases = [p for p in KW_PHRASES if p in transcript]
                        
                        if found_keywords:
                            print(f"STRESS DETECTED: Keywords {found_keywords}")
                            stress_callback()
                        
                        if found_phrases:
                            print(f"STRESS DETECTED: Phrase {found_phrases}")
                            stress_callback()
        
        except KeyboardInterrupt:
            break
        except Exception as e:
            print(f"Audio error: {e}")
            time.sleep(1)  # Avoid error spam

# Function to process sensor data from MIT App Inventor
def process_sensor_data(data):
    """Process sensor data received from the mobile app."""
    try:
        # Extract accelerometer data
        x_accel = data.get("x_accel", 0)
        y_accel = data.get("y_accel", 0)
        z_accel = data.get("z_accel", 0)
        
        # Calculate total acceleration magnitude
        total_accel = np.sqrt(x_accel**2 + y_accel**2 + z_accel**2)
        
        # Check for crash (acceleration spike greater than 2G)
        if total_accel > 2:
            print(f"CRASH DETECTED: Acceleration = {total_accel:.2f}G")
            send_to_assistant("CRASH", True)
            return True
            
        return False
    except Exception as e:
        print(f"Error processing sensor data: {e}")
        return False

# ----------------------------- MAIN -------------------------------------

def main(cam_index: int = 1):
    try:
        # Initialize camera
        cap = cv2.VideoCapture(cam_index)
        if not cap.isOpened():
            print("Camera not found - trying default camera")
            cap = cv2.VideoCapture(0)
            if not cap.isOpened():
                raise RuntimeError("No cameras found - please connect a webcam or enable camera access")

        # Initialize serial connection to Arduino
        ser = None
        try:
            ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=SERIAL_TIMEOUT)
            print(f"[ARDUINO] Connected to {SERIAL_PORT} at {BAUD_RATE} baud")
            # Give Arduino time to reset after connection
            time.sleep(2)
            # Send initial message to test connection
            send_to_arduino(ser, "SYSTEM:READY")
        except Exception as e:
            print(f"[ARDUINO] Connection failed: {e}")
            print("[ARDUINO] Continuing without Arduino communication")

        # Send system ready to assistant
        try:
            send_to_assistant("SYSTEM", True, {"message": "READY"})
            print("[ASSISTANT] Connection established")
        except Exception as e:
            print(f"[ASSISTANT] Connection failed: {e}")
            print("[ASSISTANT] Continuing without AI assistant")

        # ------- steering state -------
        cx0 = None
        steer_state = "STRAIGHT"
        last_steer_state = None
        consecutive_lr = 0

        # ------- timers & buffers -------
        eye_closed_start = None
        head_bow_start   = None
        shake_start      = None
        shake_buf        = collections.deque(maxlen=30)  # ≈1 s at 30 fps
        last_beep = 0
        
        # ------- state tracking -------
        last_drowsy = False
        last_drunk = False
        
        # ------- stress state -------
        stressed = False
        last_stressed = False
        stress_end_time = 0
        
        def trigger_stress():
            nonlocal stressed, stress_end_time
            stressed = True
            stress_end_time = time.time() + STRESS_DISPLAY_SEC
            play_beep()
        
        # Start audio processing in background thread
        audio_thread = threading.Thread(
            target=process_audio,
            args=(trigger_stress,),
            daemon=True
        )
        audio_thread.start()

        # Wait a moment for audio thread to initialize
        time.sleep(2)
        print("Driver Safety Suite started. Press SPACE to calibrate steering, Q to quit.")

        with mp_face.FaceMesh(max_num_faces=1, refine_landmarks=True,
                            min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh, \
            mp_pose.Pose(static_image_mode=False) as pose:
            
            while True:
                # Get frame from camera
                ok, frame = cap.read()
                if not ok:
                    print("Camera read failed")
                    break
                
                h, w = frame.shape[:2]
                
                # Reset stress flag if timeout elapsed
                if stressed and time.time() > stress_end_time:
                    stressed = False

                # ================== Steering detection ==================
                hsv  = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv, LOWER, UPPER)
                mask = cv2.erode(mask, None, 2)
                mask = cv2.dilate(mask, None, 2)
                blob = find_blob(mask)

                if blob and cx0 is not None:
                    dx = blob[0] - cx0
                    if abs(dx) > STEER_DEAD_PX:
                        consecutive_lr += 1
                        if consecutive_lr >= STEER_FRAMES_REQ:
                            steer_state = "LEFT" if dx < 0 else "RIGHT"
                            consecutive_lr = STEER_FRAMES_REQ
                    else:
                        consecutive_lr = 0
                        steer_state = "STRAIGHT"
                elif not blob:
                    steer_state = "STKR MISS"
                else:
                    steer_state = "UN CAL"

                cv2.putText(frame, steer_state, (20, 60), cv2.FONT_HERSHEY_DUPLEX, 1.2, (255, 255, 255), 3)

                # Send steering state to Arduino and AI assistant if changed
                if steer_state != last_steer_state and steer_state in ("LEFT", "RIGHT", "STRAIGHT"):
                    send_to_arduino(ser, f"STEER:{steer_state}")
                    send_to_assistant("STEER", True, {"direction": steer_state})
                last_steer_state = steer_state

                # ================== Face & pose landmarks ==================
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                face_res = face_mesh.process(rgb)
                pose_res = pose.process(rgb)

                drowsy = False
                drunk  = False

                # ---------- drowsiness via eyes ----------
                if face_res.multi_face_landmarks:
                    pts = np.array([(p.x * w, p.y * h) for p in face_res.multi_face_landmarks[0].landmark])
                    ear_now = (ear(pts, LIDS_L) + ear(pts, LIDS_R)) / 2

                    if ear_now < EAR_TH:
                        eye_closed_start = eye_closed_start or time.time()
                    else:
                        eye_closed_start = None

                    if eye_closed_start and time.time() - eye_closed_start >= DROWSY_SEC:
                        drowsy = True

                    # ---------- head bow ----------
                    eye_y  = pts[1][1]
                    chin_y = pts[152][1]
                    bowed = (eye_y - chin_y) > 40
                    if bowed:
                        head_bow_start = head_bow_start or time.time()
                    else:
                        head_bow_start = None
                    if head_bow_start and time.time() - head_bow_start >= BOW_SEC:
                        drunk = True

                # ---------- shake detection ----------
                if pose_res.pose_landmarks:
                    nose_x = pose_res.pose_landmarks.landmark[0].x * w
                    shake_buf.append(nose_x)
                    if len(shake_buf) == shake_buf.maxlen:
                        rms = np.std(shake_buf)
                        if rms > SHAKE_RMS_PX:
                            shake_start = shake_start or time.time()
                        else:
                            shake_start = None

                        if shake_start and time.time() - shake_start >= SHAKE_SEC:
                            drunk = True
                else:
                    shake_buf.clear()
                    shake_start = None

                # ================== Send state changes to Arduino and AI Assistant ==================
                # Send drowsy state change
                if drowsy != last_drowsy:
                    if drowsy:
                        send_to_arduino(ser, "ALERT:DROWSY")
                        send_to_assistant("DROWSY", True)
                    else:
                        send_to_arduino(ser, "CLEAR:DROWSY")
                        send_to_assistant("DROWSY", False)
                last_drowsy = drowsy
                
                # Send drunk/unwell state change
                if drunk != last_drunk:
                    if drunk:
                        send_to_arduino(ser, "ALERT:DRUNK")
                        send_to_assistant("DRUNK", True)
                    else:
                        send_to_arduino(ser, "CLEAR:DRUNK")
                        send_to_assistant("DRUNK", False)
                last_drunk = drunk
                
                # Send stress state change
                if stressed != last_stressed:
                    if stressed:
                        send_to_arduino(ser, "ALERT:STRESS")
                        send_to_assistant("STRESS", True)
                    else:
                        send_to_arduino(ser, "CLEAR:STRESS")
                        send_to_assistant("STRESS", False)
                last_stressed = stressed

                # ================== Alerts & UI ==================
                alerts = []
                if drowsy:
                    alerts.append("DROWSY")
                if drunk:
                    alerts.append("DRUNK/UNWELL")
                if stressed:
                    alerts.append("STRESS")

                for i, txt in enumerate(alerts):
                    cv2.putText(frame, txt, (20, 120 + i * 50), 
                                cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 0, 255), 3)

                if alerts and time.time() - last_beep > 0.8:
                    play_beep()
                    last_beep = time.time()
                    print(*alerts, sep=" | ")

                # Add mic status visual indicator
                mic_status = "MIC ON" if audio_thread.is_alive() else "MIC ERROR"
                cv2.putText(frame, mic_status, (w-150, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                            (0, 255, 0) if audio_thread.is_alive() else (0, 0, 255), 2)
                
                # Add Arduino status indicator
                arduino_status = "ARDUINO ON" if (ser and ser.is_open) else "ARDUINO OFF"
                cv2.putText(frame, arduino_status, (w-150, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                            (0, 255, 0) if (ser and ser.is_open) else (0, 0, 255), 2)

                # Add Assistant status indicator
                try:
                    # Quick ping to check if assistant is reachable
                    requests.get(ASSISTANT_URL.split('/alert')[0], timeout=0.1)
                    asst_status = "ASSISTANT ON"
                    asst_color = (0, 255, 0)
                except:
                    asst_status = "ASSISTANT OFF"
                    asst_color = (0, 0, 255)
                    
                cv2.putText(frame, asst_status, (w-150, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, asst_color, 2)

                # ================== Key handling & display ==================
                cv2.imshow("Driver Safety Suite", frame)
                key = cv2.waitKey(1) & 0xFF
                if key == ord(' '):
                    if blob:
                        cx0 = blob[0]
                        consecutive_lr = 0
                        print("Steering centre calibrated at", cx0)
                        send_to_arduino(ser, "SYSTEM:CALIBRATED")
                        send_to_assistant("SYSTEM", True, {"message": "CALIBRATED"})
                    else:
                        print("No sticker detected for calibration")
                elif key == ord('s'):  # Force stress test
                    trigger_stress()
                    print("Stress manually triggered")
                elif key == ord('b'):  # Test beep
                    play_beep()
                    print("Beep test")
                elif key == ord('c'):  # Test crash alert
                    print("Crash alert test")
                    send_to_assistant("CRASH", True)
                elif key == ord('q'):
                    # Send shutdown message to Arduino and assistant
                    send_to_arduino(ser, "SYSTEM:SHUTDOWN")
                    send_to_assistant("SYSTEM", False, {"message": "SHUTDOWN"})
                    break

        # Clean up
        if ser and ser.is_open:
            ser.close()
            print("[ARDUINO] Connection closed")
        
        cap.release()
        cv2.destroyAllWindows()
        
    except Exception as e:
        print(f"ERROR: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()

2025-04-22 20:59:55.455 Python[47890:1290722] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


[ARDUINO] Connected to /dev/cu.usbmodem2101 at 9600 baud
[ARDUINO] Sent: SYSTEM:READY
[ASSISTANT] Error sending to assistant: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /alert (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x321b8e1d0>: Failed to establish a new connection: [Errno 61] Connection refused'))
[ASSISTANT] Connection established


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk_en/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk_en/graph/HCLr.fst vosk_en/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo vosk_en/graph/phones/word_boundary.int


Audio processing thread started. Speech recognition: True
Calculating audio baseline... please stay quiet
Driver Safety Suite started. Press SPACE to calibrate steering, Q to quit.


I0000 00:00:1745335800.914090 1290722 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1745335800.925997 1290722 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
W0000 00:00:1745335800.930135 1291556 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745335800.947519 1291556 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745335800.963258 1291555 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
W0000 00:00:1745335801.024733 1291564 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference.

Audio baseline established: 0.045812
Audio: energy=0.040678, baseline=0.045812, ratio=0.89
